In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Post-Traumatic_Stress_Disorder/GSE52875'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression signatures in heart tissues of mice simulating posttraumatic stress disorder (PTSD)"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['strain: C57BL/6'], 1: ['tissue: Control_heart_tissue, C10R42', 'tissue: Control_heart_tissue, C10R1', 'tissue: Stressed_heart_tissue, T10R1', 'tissue: Control_heart_tissue, C5R1', 'tissue: Control_heart_tissue, C5R10', 'tissue: Stressed_heart_tissue, T10R42', 'tissue: Stressed_heart_tissue, T5R10', 'tissue: Stressed_heart_tissue, T5R1']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True  # Setting to True based on dataset inference

# Load and check keys from the sample characteristics dictionary
sample_characteristics = {0: ['strain: C57BL/6'], 1: ['tissue: heart tissue']}

trait_row = 2 if 2 in sample_characteristics else None
age_row = 3 if 3 in sample_characteristics else None
gender_row = 4 if 4 in sample_characteristics else None

# Defining the conversion functions

def convert_trait(value):
    try:
        return int(value.split(':')[1].strip())
    except:
        return None

def convert_age(value):
    try:
        return float(value.split(':')[1].strip())
    except:
        return None

def convert_gender(value):
    try:
        gender = value.split(':')[1].strip().lower()
        if gender == 'female':
            return 0
        elif gender == 'male':
            return 1
    except:
        return None

# Saving cohort information
save_cohort_info('GSE52875', './preprocessed/Post-Traumatic_Stress_Disorder/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction if applicable
if trait_row is not None:
    try:
        selected_clinical_data = geo_select_clinical_features(clinical_data, 'Post-Traumatic_Stress_Disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
        csv_path = './preprocessed/Post-Traumatic_Stress_Disorder/trait_data/GSE52875.csv'
        selected_clinical_data.to_csv(csv_path)
        print(preview_df(selected_clinical_data))
    except Exception as e:
        print(f"Error in clinical feature extraction: {e}")
